In [1]:
#!pip install pomegranate

In [2]:
#!pip install pymc3

In [3]:
#!pip install graphviz

In [4]:
#!pip install pygraphviz

In [5]:
#!pip install matplotlib

In [6]:
#!pip install openpyxl

In [7]:
#!pip install sklearn

In [8]:
#imports
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt
from pomegranate import *

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\powel\Anaconda3\lib\site-packages\theano\configdefaults.py:697: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  "DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [9]:
#read in data
pd.set_option('display.max_columns', None)

#name format is df survey#_wave# if resend, add (_2 at end)

df1_1 = pd.read_csv("survey1.1.csv", encoding='cp1252')
df1_1_2 = pd.read_csv("survey1.1.2.csv", encoding='cp1252')
df1_2 = pd.read_csv("survey1.2.csv", encoding ='cp1252')
df1_3 = pd.read_csv("survey1.3.csv", encoding ='cp1252')
#survey 1 wave 3
#survey1.3.csv
#df1_3 = pd.read_csv...
df2_1 = pd.read_csv("survey2.1.csv", encoding='cp1252')
df2_1_2 = pd.read_csv("survey2.1.2.csv", encoding='cp1252')
df2_2 = pd.read_csv("survey2.2.csv", encoding='cp1252')
df2_3 = pd.read_csv("survey2.3.csv", encoding='cp1252')
df3_1 = pd.read_csv("survey3.1.csv", encoding='cp1252')
df3_2 = pd.read_csv("survey3.2.csv", encoding='cp1252')

In [10]:
def conditionalDrop(columns, df):
    for x in columns:
        if x in df.columns:
            df=df.drop(columns = x)
            
    return df

In [11]:
DropWave3 = ['Sense of Wellbeing',
'Sense of Hope',
'Conditional Question D or SD',
'Caregive Support Programs Participation',
'Other Caregiver Engagement',
'Continue Participation',
'MVCN Support Programs and Services',
'MVCN Volunteer',
'MVCN Program Participation',
'Caregiver Community Connectedness']
#df3_2_1 = conditionalDrop(DropWave3, df3_2)
df2_3_1 = conditionalDrop(DropWave3, df2_3)
df1_3_1 = conditionalDrop(DropWave3, df1_3)
df2_3_2 = df2_3[DropWave3]
df1_3_2 = df1_3[DropWave3]
df2_3_2 = conditionalDrop(['Conditional Question D or SD'], df2_3_2)
df1_3_2 = conditionalDrop(['Conditional Question D or SD'], df1_3_2)

In [12]:
df1_3_2

,Sense of Wellbeing,Sense of Hope,Caregive Support Programs Participation,Other Caregiver Engagement,Continue Participation,MVCN Support Programs and Services,MVCN Volunteer,MVCN Program Participation,Caregiver Community Connectedness
0,Did NOT Participate,Did NOT Particpiate,Neutral,Neutral,Neutral,No,No,No,Agree
1,Neutral,Neutral,Strongly Disagree,Strongly Disagree,Neutral,No,No,No,Neutral
2,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Yes,No,Yes,Agree
3,Agree,Agree,Agree,Agree,Agree,Yes,No,Yes,Strongly Agree
4,Agree,Agree,Neutral,Neutral,Agree,Yes,No,Unsure,Agree
...,...,...,...,...,...,...,...,...,...
304,Agree,Agree,Agree,Neutral,Agree,Yes,No,Yes,Agree
305,Agree,Agree,Agree,Neutral,Agree,Unsure,No,Yes,Agree
306,Neutral,Neutral,Neutral,Neutral,Neutral,No,No,No,Neutral
307,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Yes,No,Yes,Strongly Agree


In [13]:
#DropSurvey3 = ['Video Conference (Connection Technology Use)',
#'Race',
#'Service 9/11',
#'Spouse/Partner/Significant Other (Relationship)',
#'Child (Relationship)',
#'Sibling (Relationship)',
#'Parent (Relationship)',
#'Extended Family Member (Relationship)',
#'Friend (Relationship)',
#'Other (Relationship)',
#'Work Status']
#df3_2 = df3_2.drop(columns = DropSurvey3)

In [14]:
survey1_1 = pd.concat([df1_1, df1_1_2, df1_2], ignore_index=True)#Normal wave 1 and 2 data
survey2_1 = pd.concat([df2_1, df2_1_2, df2_2], ignore_index=True)
survey1_1['survey']=np.full(survey1_1.shape[0], 1)
survey2_1['survey']=np.full(survey2_1.shape[0], 2)
main_1 = pd.concat([survey1_1, survey2_1], ignore_index=True)

In [15]:
df1_3_1['survey']=np.full(df1_3_1.shape[0], 1)#Wave 3 without the wave 3 anly questions
df2_3_1['survey']=np.full(df2_3_1.shape[0], 2)
main_2 = pd.concat([df1_3_1, df2_3_1], ignore_index=True)

In [16]:
df1_3_2['survey']=np.full(df1_3_2.shape[0], 1)#Wave 3 with only the wave 3 anly questions
df2_3_2['survey']=np.full(df2_3_2.shape[0], 2)
main_3 = pd.concat([df1_3_2, df2_3_2], ignore_index=True)

In [17]:
print(survey1_1.shape)
print(survey2_1.shape)
print(df1_3_1.shape)
print(df2_3_1.shape)

(702, 71)
(605, 71)
(309, 58)
(312, 68)


In [18]:
toDrop = ['Time Providing Care','Use of Video Conferencing', 'Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent', 'Flow', 'Display and Format', 'Content',]
main_1 = conditionalDrop(toDrop, main_1)
main_2 = conditionalDrop(toDrop, main_2)

In [19]:
main_2.drop(list(main_2.filter(regex = 'Constituent Type')), axis = 1, inplace = True) # not present in wave 3
main_2 = conditionalDrop(['Sex', 'Age', 'Race:',], main_2) 

In [20]:
#main_1 = main_1.dropna()
#main_2 = main_2.dropna()
#main_3 = main_3.dropna()

In [21]:
main_1

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,15,0,1,0,0,0,Female,45 - 54,White,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree,1
1,14,0,1,0,0,0,Female,35 - 44,White,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree,1
2,15,0,0,0,1,0,Male,35 - 44,White,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree,1
3,12,0,0,0,1,0,Male,65+,White,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree,1
4,15,0,0,0,1,0,Male,35 - 44,Black or African American,Yes,Yes,Strongly Disagree,Strongly Agree,Strongly Disagree,Neutral,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Sometimes,Rarely,Rarely,Rarely,Sometimes,Always,Rarely,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,Agree,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,15,0,1,0,0,0,Female,25 - 34,White,No,Yes,Strongly Disagree,Strongly Agree,Disagree,Agree,Neutral,Agree,Neutral,Strongly Disagree,Disagree,Agree,Neutral,Neutral,Agree,Agree,Rare

In [22]:
main_2

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,0,No,No,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,1,0,Agree,1
1,0,No,Yes,Agree,Agree,Disagree,Neutral,Strongly Agree,Neutral,Neutral,Neutral,Disagree,Neutral,Agree,Neutral,Neutral,Neutral,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Never,Never,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,Disagree,1
2,0,No,No,Strongly Disagree,Agree,Agree,Agree,Agree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Agree,Never,Sometimes,Sometimes,Sometimes,Never,Sometimes,Sometimes,Never,Sometimes,Never,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,Neutral,1
3,0,Yes,Yes,Neutral,Agree,Agree,Disagree,Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,1,1,1,0,1,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,Strongly Agree,1
4,0,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Strongly Disagree,Neutral,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Agree,Rarely,Never,Never,Never,Rarely,Never,Never,Never,Sometimes,Rarely,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,Agree,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,0,No,No,Disagree,Neutral,Strongly Disagree,Strongly Disagree,Neutral,Agree,Agree,Neutral,Strongly Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Rarely,Rarely,Sometimes,Sometimes,Rarely,Rarely,Sometimes,Sometimes,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,1,1,1,0,Strongly Agree,2
617,0,Yes,Yes,Agree,Agree,Neutral,Neutral,Strongly Agree,Agree,Strongly Agree,Neutral,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Always,Sometimes,Sometimes,Sometimes,Always,Sometimes,Always,Always,Always,Rarely,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,Strongly Disagree,2
618,0,No,Yes,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Strong

In [23]:
main_3

,Sense of Wellbeing,Sense of Hope,Caregive Support Programs Participation,Other Caregiver Engagement,Continue Participation,MVCN Support Programs and Services,MVCN Volunteer,MVCN Program Participation,Caregiver Community Connectedness,survey
0,Did NOT Participate,Did NOT Particpiate,Neutral,Neutral,Neutral,No,No,No,Agree,1
1,Neutral,Neutral,Strongly Disagree,Strongly Disagree,Neutral,No,No,No,Neutral,1
2,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Yes,No,Yes,Agree,1
3,Agree,Agree,Agree,Agree,Agree,Yes,No,Yes,Strongly Agree,1
4,Agree,Agree,Neutral,Neutral,Agree,Yes,No,Unsure,Agree,1
...,...,...,...,...,...,...,...,...,...,...
616,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Yes,No,No,Strongly Agree,2
617,Strongly Agree,Agree,Agree,Neutral,Strongly Agree,No,No,Yes,Strongly Agree,2
618,Did NOT Participate,Did NOT Particpiate,Neutral,Neutral,Neutral,No,No,No,Neutral,2
619,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Yes,No,No,Strongly Agree,2


In [24]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'Unsure': 0,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '<6mo':-3,
    '6mo to 1year':-2,
    '1 to 2 years': -1,
    '2 to 3 years': 0,
    '3 to 4 years':1,
    '4-5 years': 2,
    '5+ years': 3, 
    '<3 Months': -2,
    '3 To 6 Months': -1,
    '6 To 9 Months': 0,
    '9 To 12 Months': 1,
    '1+ Year': 2,
    '0 - 17': -3,
    '18 - 24': -2,
    '25 - 34': -1,
    '35 - 44': 0,
    '45 - 54': 1,
    '55 - 64': 2,
    '65+': 3,
    'Male': 0,
    'Female': 1,
    'White': -2, 
    'Black or African American': -1,
    'Asian': 0,
    'American Indian or Alaska Native': 1,
    'Some other race': 2,
    'Native Hawaiian and Pacific Islander': 3,
    'Did NOT Participate': -10,
    'Did NOT Particpiate': -10
}
def f(s):
    l = []
    for b in s:
        #print(b)
        if (b in encoding):
            l.append(encoding[b])
        elif isinstance(b, str) and b.isnumeric():
            l.append(int(b))
        elif np.isnan(b):
            l.append(int(-10))
        else:
            l.append(b)
    return pd.Series(l)


In [25]:
data_1 = main_1.apply(f, axis = 0)
data_2 = main_2.apply(f, axis = 0)
data_3 = main_3.apply(f, axis = 0)

In [26]:
data_1

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,15,0,1,0,0,0,1,1,-2,1,-1,0,1,-1,1,2,2,2,0,-2,1,2,2,2,1,-1,1,-2,-2,1,1,-1,1,1,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,1,1
1,14,0,1,0,0,0,1,0,-2,-1,-1,-2,-1,-2,-1,-1,-2,-2,-1,-2,-1,1,1,1,0,1,1,1,-1,-2,-2,-2,-2,-1,-1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,1,1
2,15,0,0,0,1,0,0,0,-2,-1,-1,-2,-2,-2,0,2,-2,-2,-2,-2,-2,0,0,1,1,-2,-2,-1,-2,-2,-2,-2,-2,-2,-2,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1
3,12,0,0,0,1,0,0,3,-2,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1
4,15,0,0,0,1,0,0,0,-1,1,1,-2,2,-2,0,-2,-2,-2,-2,-2,0,2,2,1,1,1,-1,1,1,-1,-1,-1,1,2,-1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,15,0,1,0,0,0,1,-1,-2,-1,1,-2,2,-1,1,0,1,0,-2,-1,1,0,0,1,1,-1,-1,-2,-2,-2,-2,-2,-2,2,2,0,0,1,1,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,2
1303,15,0,0,1,1,0,0,2,-2,1,1,1,2,1,1,0,1,1,1,2,1,1,1,1,1,1,1,-1,1,1,1,1,1,2,1,1,1,0,1,1,0,0,0,0,1,0,1,1,0,1,1,1,1,0,1,1,1,0,1,2
1304,15,0,1,0,0,0,1,1,-2,1,-1,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,2,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,2
1305,15,0,0,0,1,0,1,0,-1,1,1,0,1,1,1,1,0,2,2,1,1,2,1,1,1,1,1,1,1,2,1,1,1,2,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,0,2,2


In [27]:
data_2

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,-1,1,-1,-1,-1,-1,1,1,-1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,1,0,1,1
1,0,-1,1,1,1,-1,0,2,0,0,0,-1,0,1,0,0,0,1,1,-1,-1,-1,-1,-1,1,-2,-2,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,-1,1
2,0,-1,-1,-2,1,1,1,1,-1,-1,-1,-1,1,1,1,1,1,-2,1,1,1,-2,1,1,-2,1,-2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1
3,0,1,1,0,1,1,-1,-1,2,2,2,1,1,2,2,2,2,1,1,-1,-1,-1,-1,-1,1,1,-1,1,1,1,0,1,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,2,1
4,0,-1,-1,-2,-1,-2,-1,1,-2,0,-1,-2,-1,1,1,1,1,-1,-2,-2,-2,-1,-2,-2,-2,1,-1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,0,-1,-1,-1,0,-2,-2,0,1,1,0,2,0,2,2,2,2,1,1,-1,-1,1,1,-1,-1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,1,1,1,0,2,2
617,0,1,1,1,1,0,0,2,1,2,0,1,2,2,2,1,1,2,1,1,1,2,1,2,2,2,-1,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,-2,2
618,0,-1,1,2,2,2,2,-1,2,2,2,2,2,1,-1,0,0,1,1,1,1,-1,1,1,-1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,2
619,0,-1,-1,-1,0,-2,-2,0,1,1,0,2,0,2,2,2,2,1,1,-1,-1,1,1,-1,-1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,1,1,1,0,2,2


In [28]:
data_3

,Sense of Wellbeing,Sense of Hope,Caregive Support Programs Participation,Other Caregiver Engagement,Continue Participation,MVCN Support Programs and Services,MVCN Volunteer,MVCN Program Participation,Caregiver Community Connectedness,survey
0,-10,-10,0,0,0,-1,-1,-1,1,1
1,0,0,-2,-2,0,-1,-1,-1,0,1
2,2,2,1,1,2,1,-1,1,1,1
3,1,1,1,1,1,1,-1,1,2,1
4,1,1,0,0,1,1,-1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
616,2,2,2,2,2,1,-1,-1,2,2
617,2,1,1,0,2,-1,-1,1,2,2
618,-10,-10,0,0,0,-1,-1,-1,0,2
619,2,2,2,2,2,1,-1,-1,2,2


In [29]:
temp_1 = data_1.copy()
temp_1.drop(list(temp_1.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp_1.drop(list(temp_1.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
temp_1 = conditionalDrop(['Sex', 'Age', 'Race:','survey'], temp_1)

temp_2 = data_2.copy()
temp_2.drop(list(temp_2.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp_2 = conditionalDrop(['Sex', 'Age', 'Race:','survey'], temp_2)


In [30]:
temp_1

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help
0,15,1,-1,0,1,-1,1,2,2,2,0,-2,1,2,2,2,1,-1,1,-2,-2,1,1,-1,1,1,1,1,1,1,0,0,1,1,1,0,1
1,14,-1,-1,-2,-1,-2,-1,-1,-2,-2,-1,-2,-1,1,1,1,0,1,1,1,-1,-2,-2,-2,-2,-1,-1,1,1,1,0,0,1,1,0,0,1
2,15,-1,-1,-2,-2,-2,0,2,-2,-2,-2,-2,-2,0,0,1,1,-2,-2,-1,-2,-2,-2,-2,-2,-2,-2,0,1,0,0,0,0,1,0,0,1
3,12,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,1,0,1
4,15,1,1,-2,2,-2,0,-2,-2,-2,-2,-2,0,2,2,1,1,1,-1,1,1,-1,-1,-1,1,2,-1,0,1,1,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,15,-1,1,-2,2,-1,1,0,1,0,-2,-1,1,0,0,1,1,-1,-1,-2,-2,-2,-2,-2,-2,2,2,1,1,1,0,0,1,1,1,0,1
1303,15,1,1,1,2,1,1,0,1,1,1,2,1,1,1,1,1,1,1,-1,1,1,1,1,1,2,1,1,1,1,1,0,1,1,1,0,1
1304,15,1,-1,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,2,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0
1305,15,1,1,0,1,1,1,1,0,2,2,1,1,2,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,0,0,1,1,1,0,2


In [31]:
data_12 = pd.concat([data_1, data_2], ignore_index=True)
temp_12 = pd.concat([temp_1, temp_2], ignore_index=True)

In [32]:

import statsmodels.stats.weightstats as ssw

In [33]:
def pval(temp, data1, data2, name): #calculates p-value
    results = []

    for col in temp.columns: 
        vals1 = data1[col].to_list()
        vals2 = data2[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
        cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
        res = cm.ztest_ind(alternative='smaller',usevar='unequal')
        msg = "none"
        alpha = 0.05/39
        if (res[1] <= alpha):
            msg = "larger"
       
        elif (res[1] >= 1-alpha):
            msg = "smaller"
    
        if (res[1] <= 0.5):
            results.append([res[1]])
        else:
            results.append([res[1]-1])

    res = pd.DataFrame(results, columns=[name])
    return res 

In [34]:
def esize(temp, data1, data2, name): #calculates effective size
    results = []

    for col in temp.columns: 
        vals1 = data1[col].to_list()
        vals2 = data2[col].to_list()
        
        mean1 = sum(vals1)/len(vals1)
        mean2 = sum(vals2)/len(vals2)
        diff = mean2 - mean1 
        msg = "none"
        if(diff >= 0):
            msg = "larger"
        else:
            msg = "smaller"
            
        results.append([diff])

    res = pd.DataFrame(results, columns=[name])
    return res 

In [35]:
pd.set_option('display.float_format', lambda x: '{:e}'.format(x))

In [36]:
PSurveyComps = pd.DataFrame(temp_1.columns, columns=['Item'])
PItemComps = pd.DataFrame(temp_1.columns, columns=['Item'])
ESurveyComps = pd.DataFrame(temp_1.columns, columns=['Item'])
EItemComps = pd.DataFrame(temp_1.columns, columns=['Item'])

In [37]:
pd.concat([PSurveyComps, pval(temp_1, temp_1[data_1['survey']==1], temp_1[data_1['survey']==2], "All")], axis = 1)
pd.concat([ESurveyComps, esize(temp_1, temp_1[data_1['survey']==1], temp_1[data_1['survey']==2], "All")], axis = 1)


,Item,All
0,Score (Responses to recipient forms),9.428716e-01
1,Depression,1.355278e-02
2,Motivation,-4.127522e-02
3,Life Satisfaction,1.959266e-01
4,Physical Exhaustion,1.257847e-01
5,Desire To Escape,1.198677e-01
6,Same Self,8.554543e-02
7,Financial Decrease,1.322126e-01
8,Health Change,2.001319e-01
9,Demands Strength,2.277060e-01


In [38]:
def stats3(low, high, item, temp, data, name1, name2, name3):
    a1data = temp[data[item]<= low]
    a2data = temp[data[item]>= high]
    
    b1data = a1data[data['survey'] == 1]
    b2data = a1data[data['survey'] == 2]
 
    
    c1data = a2data[data['survey'] == 1]
    c2data = a2data[data['survey'] == 2]

    return [pval(temp, a1data, a2data, name1), [pval(temp, b1data, b2data, name2), pval(temp, c1data, c2data, name3)],
            esize(temp, a1data, a2data, name1), [esize(temp, b1data, b2data, name2), esize(temp, c1data, c2data, name3)]]

In [39]:
a = stats3(0, 1, 'Age', temp_1, data_1, 'Older vs Younger Than 44', 'Younger Than 44', 'Older Than 44')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'Sex', temp_1, data_1, 'Males vs Females', 'Males', 'Females')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(-2, -1, 'Race:', temp_1, data_1, 'Whites vs Non-Whites', 'Whites', 'Non-Whites')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'Veteran (Constituent Type)', temp_1, data_1, 'Caretakers vs Veterans', 'Caretakers', 'Veterans')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

diseases = ['Hypertension (high blood pressure) (Chronic Conditions)','Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)','Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)','Arthritis (Chronic Conditions)','Diabetes (Type1 and Type 2)  (Chronic Conditions)', 'Mood Disorder (depression and bipolar disorder) (Chronic Conditions)','Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions)','Anxiety Disorder (Chronic Conditions)','Asthma (Chronic Conditions)','Thyroid disorder (Chronic Conditions)','Traumatic Brain Injury (TBI) (Chronic Conditions)']
for x in diseases:
    a = stats3(0, 1, x, temp_12, data_12, 'Does Not Have vs Has ' + x.split('(')[0], 'Does Not Have ' + x.split('(')[0], 'Has ' + x.split('(')[0])
    PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
    PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
    EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
    ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [40]:
MVCN = ['MVCN Support Programs and Services', 'MVCN Volunteer', 'MVCN Program Participation']

a = stats3(0, 1, 'MVCN Support Programs and Services', temp_2, data_3, 'Not Member vs Member of MVCN Support Programs', 'Not Member of MVCN Support Programs', 'Member of MVCN Support Programs')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'MVCN Volunteer', temp_2, data_3, 'Is vs Is Not MVCN Volunteer', 'Is Not MVCN Volunteer', 'Is MVCN Volunteer')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'MVCN Program Participation', temp_2, data_3, 'Does Not vs Does Participate in MVCN Program', 'Does Not Participate in MVCN Program', 'Participates in MVCN Program')
PItemComps = pd.concat([PItemComps, a[0]], axis = 1)
PSurveyComps = pd.concat([PSurveyComps, a[1][0],a[1][1]], axis = 1)
EItemComps = pd.concat([EItemComps, a[2]], axis = 1)
ESurveyComps = pd.concat([ESurveyComps, a[3][0], a[3][1]], axis = 1)

C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [41]:
PItemComps

,Item,Older vs Younger Than 44,Males vs Females,Whites vs Non-Whites,Caretakers vs Veterans,Does Not Have vs Has Hypertension,Does Not Have vs Has Hyperlipidemia,"Does Not Have vs Has Allergies, sinusitis and other upper respiratory conditions",Does Not Have vs Has Arthritis,Does Not Have vs Has Diabetes,Does Not Have vs Has Mood Disorder,Does Not Have vs Has Post-Traumatic Stress Disorder,Does Not Have vs Has Anxiety Disorder,Does Not Have vs Has Asthma,Does Not Have vs Has Thyroid disorder,Does Not Have vs Has Traumatic Brain Injury,Not Member vs Member of MVCN Support Programs,Is vs Is Not MVCN Volunteer,Does Not vs Does Participate in MVCN Program
0,Score (Responses to recipient forms),6.407840e-02,1.503832e-02,4.939055e-01,-2.337915e-03,-3.173331e-01,-3.727416e-01,-9.922560e-02,1.167048e-01,4.805077e-01,-2.376860e-01,-3.662263e-01,-3.000831e-01,3.927122e-01,-1.764331e-01,-5.364492e-02,nan,nan,nan
1,Depression,-6.282936e-02,-1.801438e-05,-8.416261e-02,1.364595e-10,-1.174201e-01,1.623412e-02,8.555760e-06,8.292640e-02,3.898246e-01,2.695256e-34,1.423113e-17,8.519337e-27,1.642858e-01,-3.741596e-01,5.534123e-04,1.152887e-01,3.714917e-01,9.021066e-02
2,Motivation,-1.526732e-02,-6.679080e-06,-2.643954e-01,1.662314e-12,-1.318710e-01,5.486299e-02,1.442597e-10,1.057427e-01,-2.925977e-01,9.346106e-36,4.038521e-21,1.099058e-27,1.367336e-02,3.493160e-01,1.388665e-06,1.311130e-01,2.405077e-01,1.601761e-01
3,Life Satisfaction,1.056282e-01,1.962918e-09,-4.259518e-05,-2.625019e-08,1.950038e-01,2.321670e-02,1.909394e-01,-4.928127e-01,-2.158008e-01,1.422746e-06,4.852986e-03,3.268540e-09,-9.053960e-02,8.529193e-02,3.218508e-02,-3.641566e-01,2.304204e-01,3.450681e-01
4,Physical Exhaustion,-1.273992e-01,1.578893e-01,-4.489460e-05,2.182824e-02,-3.526317e-01,1.444231e-03,2.408281e-06,3.581107e-04,4.192356e-01,1.026228e-19,1.528614e-08,2.331588e-23,2.834772e-03,1.242231e-01,6.477996e-02,-7.609422e-02,-8.779682e-02,4.901220e-01
5,Desire To Escape,6.150418e-02,-7.255997e-02,-9.074895e-06,4.361062e-05,-8.904810e-02,2.400951e-01,1.103716e-01,4.590452e-01,-3.772079e-01,3.355781e-10,1.711294e-03,3.582461e-11,-1.113381e-01,1.601003e-01,2.045556e-01,2.567928e-01,3.611243e-01,2.385511e-02
6,Same Self,-2.171235e-01,-4.081445e-03,-3.664596e-03,1.685958e-08,-1.574447e-01,1.057398e-01,1.614961e-04,9.924695e-02,-1.609437e-01,7.476356e-19,2.368022e-09,1.668122e-23,1.052496e-01,2.443323e-01,7.104791e-04,3.140791e-01,-6.200121e-02,7.050037e-02
7,Financial Decrease,3.370092e-01,1.021206e-03,-3.136384e-02,-1.192699e-02,3.989974e-01,2.644067e-01,1.777622e-03,2.302973e-01,9.628494e-02,4.457734e-05,1.459643e-02,1.949314e-06,7.147021e-04,1.884508e-01,2.475942e-01,6.503439e-02,-4.801402e-01,5.177827e-02
8,Health Change,6.756348e-03,1.290067e-04,-1.726092e-04,-6.000665e-02,3.124006e-03,2.449463e-06,5.925265e-06,1.149149e-04,4.400579e-01,5.702136e-08,7.677558e-03,2.461445e-13,2.012070e-02,3.088272e-03,4.125246e-01,2.987043e-01,-1.239180e-01,8.858583e-02
9,Demands Strength,3.783779e-04,3.653587e-06,-2.931342e-04,-1.157469e-03,7.231823e-02,1.233340e-03,1.655633e-05,1.707303e-04,3.784134e-01,1.106645e-04,4.686680e-02,2.187749e-07,1.177101e-01,1.344804e-02,-1.408973e-01,-1.376849e-01,-8.663513e-03,4.096837e-01


In [71]:
data_3 = data_3.dropna()
temp_3 = data_3[['Sense of Wellbeing',
'Sense of Hope',
'Caregive Support Programs Participation',
'Continue Participation',
'Caregiver Community Connectedness']]

In [72]:
Wave3_PSurveyComps = pd.DataFrame(temp_3.columns, columns=['Item'])
Wave3_PItemComps = pd.DataFrame(temp_3.columns, columns=['Item'])
Wave3_ESurveyComps = pd.DataFrame(temp_3.columns, columns=['Item'])
Wave3_EItemComps = pd.DataFrame(temp_3.columns, columns=['Item'])

In [73]:
diseases = ['Hypertension (high blood pressure) (Chronic Conditions)','Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)','Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)','Arthritis (Chronic Conditions)','Diabetes (Type1 and Type 2)  (Chronic Conditions)', 'Mood Disorder (depression and bipolar disorder) (Chronic Conditions)','Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions)','Anxiety Disorder (Chronic Conditions)','Asthma (Chronic Conditions)','Thyroid disorder (Chronic Conditions)','Traumatic Brain Injury (TBI) (Chronic Conditions)']
for x in diseases:
    a = stats3(0, 1, x, temp_3, data_2, 'Does Not Have vs Has ' + x.split('(')[0], 'Does Not Have ' + x.split('(')[0], 'Has ' + x.split('(')[0])
    Wave3_PItemComps = pd.concat([Wave3_PItemComps, a[0]], axis = 1)
    Wave3_PSurveyComps = pd.concat([Wave3_PSurveyComps, a[1][0],a[1][1]], axis = 1)
    Wave3_EItemComps = pd.concat([Wave3_EItemComps, a[2]], axis = 1)
    Wave3_ESurveyComps = pd.concat([Wave3_ESurveyComps, a[3][0], a[3][1]], axis = 1)

C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [74]:
a = stats3(0, 1, 'MVCN Support Programs and Services', temp_3, data_3, 'Not Member vs Member of MVCN Support Programs', 'Not Member of MVCN Support Programs', 'Member of MVCN Support Programs')
Wave3_PItemComps = pd.concat([Wave3_PItemComps, a[0]], axis = 1)
Wave3_PSurveyComps = pd.concat([Wave3_PSurveyComps, a[1][0],a[1][1]], axis = 1)
Wave3_EItemComps = pd.concat([Wave3_EItemComps, a[2]], axis = 1)
Wave3_ESurveyComps = pd.concat([Wave3_ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'MVCN Volunteer', temp_3, data_3, 'Is vs Is Not MVCN Volunteer', 'Is Not MVCN Volunteer', 'Is MVCN Volunteer')
Wave3_PItemComps = pd.concat([Wave3_PItemComps, a[0]], axis = 1)
Wave3_PSurveyComps = pd.concat([Wave3_PSurveyComps, a[1][0],a[1][1]], axis = 1)
Wave3_EItemComps = pd.concat([Wave3_EItemComps, a[2]], axis = 1)
Wave3_ESurveyComps = pd.concat([Wave3_ESurveyComps, a[3][0], a[3][1]], axis = 1)

a = stats3(0, 1, 'MVCN Program Participation', temp_3, data_3, 'Does Not vs Does Participate in MVCN Program', 'Does Not Participate in MVCN Program', 'Participates in MVCN Program')
Wave3_PItemComps = pd.concat([Wave3_PItemComps, a[0]], axis = 1)
Wave3_PSurveyComps = pd.concat([Wave3_PSurveyComps, a[1][0],a[1][1]], axis = 1)
Wave3_EItemComps = pd.concat([Wave3_EItemComps, a[2]], axis = 1)
Wave3_ESurveyComps = pd.concat([Wave3_ESurveyComps, a[3][0], a[3][1]], axis = 1)

C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\powel\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [75]:
Wave3_ESurveyComps

,Item,Does Not Have Hypertension,Has Hypertension,Does Not Have Hyperlipidemia,Has Hyperlipidemia,"Does Not Have Allergies, sinusitis and other upper respiratory conditions","Has Allergies, sinusitis and other upper respiratory conditions",Does Not Have Arthritis,Has Arthritis,Does Not Have Diabetes,Has Diabetes,Does Not Have Mood Disorder,Has Mood Disorder,Does Not Have Post-Traumatic Stress Disorder,Has Post-Traumatic Stress Disorder,Does Not Have Anxiety Disorder,Has Anxiety Disorder,Does Not Have Asthma,Has Asthma,Does Not Have Thyroid disorder,Has Thyroid disorder,Does Not Have Traumatic Brain Injury,Has Traumatic Brain Injury,Not Member of MVCN Support Programs,Member of MVCN Support Programs,Is Not MVCN Volunteer,Is MVCN Volunteer,Does Not Participate in MVCN Program,Participates in MVCN Program
0,Sense of Wellbeing,-3.123321e-01,-6.455803e-01,-2.606153e-01,-9.693122e-01,-5.864935e-01,-3.561502e-01,-9.304330e-01,1.236068e-01,-3.615585e-01,-7.505411e-01,-2.440945e-01,-6.053191e-01,-2.544643e-01,-5.886718e-01,-2.295198e-01,-5.989778e-01,-5.415284e-01,-8.771930e-02,-5.896075e-01,2.155556e-01,-1.103523e-01,-9.897959e-01,-4.567839e-01,-1.308225e-01,-4.921698e-01,-1.307692e-01,-2.712418e-01,-2.418415e-01
1,Sense of Hope,2.958893e-02,-4.179012e-01,-1.188660e-01,-4.486772e-01,-4.355325e-01,3.095762e-02,-6.544507e-01,3.736177e-01,-1.322381e-01,-3.612915e-01,-2.039435e-01,-1.858539e-01,2.385913e-01,-4.334393e-01,1.213512e-01,-3.925767e-01,-3.141277e-01,3.010526e-01,-3.739704e-01,6.933333e-01,2.983189e-01,-9.561224e-01,-1.338079e-01,6.825821e-02,-2.118093e-01,-1.384615e-02,8.398693e-02,-5.885781e-02
2,Caregive Support Programs Participation,2.845747e-01,-4.489968e-01,-9.210526e-02,-3.068783e-02,-4.508052e-01,2.874243e-01,-4.240176e-01,3.561344e-01,-7.097585e-02,-1.659452e-02,-1.695842e-01,1.175963e-02,-3.239087e-01,9.223080e-02,-9.733992e-02,2.465096e-02,-1.605329e-01,2.610526e-01,-2.175515e-01,4.894444e-01,1.891003e-01,-4.795918e-01,5.912898e-02,-1.008438e-02,-5.257067e-02,-1.723077e-01,2.545752e-01,-1.392774e-01
3,Continue Participation,-5.677656e-02,-4.061124e-01,-3.040936e-01,1.657848e-02,-3.647273e-01,1.066318e-02,-7.835204e-01,4.557892e-01,-1.227040e-01,-3.410895e-01,-1.804516e-01,-1.746406e-01,-2.827381e-01,-1.232016e-01,-3.495763e-01,3.178758e-03,-2.442433e-01,8.561404e-02,-3.395109e-01,3.844444e-01,-3.506055e-02,-3.428571e-01,-1.174204e-01,-4.699184e-02,-1.748498e-01,-2.184615e-01,4.901961e-02,-1.701632e-01
4,Caregiver Community Connectedness,2.832316e-01,-3.129588e-01,3.318078e-02,1.481481e-02,-5.651948e-02,1.661392e-01,-5.080593e-01,6.526509e-01,3.041566e-02,1.098485e-01,-1.626863e-02,9.554342e-02,-1.413690e-01,1.475844e-01,-8.015537e-02,1.760160e-01,8.154852e-02,-4.842105e-02,-7.701094e-02,4.833333e-01,2.749267e-01,-3.469388e-01,2.089336e-01,-1.152501e-02,4.872944e-02,6.000000e-02,3.794118e-01,-8.799534e-02


In [76]:

PSurveyComps.to_csv('PSurveyComps.csv')
PItemComps.to_csv('PItemComps.csv')
ESurveyComps.to_csv('ESurveyComps.csv')
EItemComps.to_csv('EItemComps.csv')

Wave3_PSurveyComps.to_csv('Wave3_PSurveyComps.csv')
Wave3_PItemComps.to_csv('Wave3_PItemComps.csv')
Wave3_ESurveyComps.to_csv('Wave3_ESurveyComps.csv')
Wave3_EItemComps.to_csv('Wave3_EItemComps.csv')

